In [0]:
# Initial Steps
# Don't forget to change the runtime to GPU

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#path 
import os
user_id = 2

if user_id==0: # Chirag
  os.chdir('/content/drive/My Drive/IIITD_Course/Sem_6/DL/DL_Group/Assignments/Assignment_1')
elif user_id==1: # Prakhar
  os.chdir('/content/drive/My Drive/IIITD_Course/Sem_6/DL/DL_Group/Assignments/Assignment_1')
elif user_id==2: # Akash
  os.chdir('/content/drive/My Drive/DL_Group/Assignments/Assignment_1/Datasets/Q3')

In [0]:
#importing libraries

import json
import csv

import collections
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
from torchvision import transforms
from retinanet import model
from retinanet.dataloader import CocoDataset, CSVDataset, collater, Resizer, AspectRatioBasedSampler, Augmenter, \
    Normalizer
from torch.utils.data import DataLoader
from retinanet import coco_eval
from retinanet import csv_eval

In [0]:
!apt-get install tk-dev python-tk
!pip install pycocotools

Reading package lists... Done
Building dependency tree       
Reading state information... Done
tk-dev is already the newest version (8.6.0+9).
python-tk is already the newest version (2.7.17-1~18.04).
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.


In [0]:
#cleaning the annotations json file -> removing other classes & invalid bounding box 
# bounding box dimensions are in int
mainIds = [11,6,33,16,3,10,9,5] # IDs of given classes
mainClasses = ["bird","bobcat","car","cat","raccoon","rabbit","coyote","squirrel"]

with open('train_annotations.json') as data_file:    
	data = json.load(data_file)
	# print(data['categories'])
	# print(data['info'])
	# print(data['images'])
	x=data['annotations']
	y=[]
	for v in x:
			
			if(v['category_id'] in mainIds):
				classs=mainClasses[mainIds.index(v['category_id'])]
				pathe="train/"+classs+"/"+v['image_id']+".jpg"
				a=True
				try:
					qq=v['bbox']
				except:
					a=False
				if(a==True):
					if(int(v['bbox'][2])>int(v['bbox'][0])):
						if(int(v['bbox'][3])>int(v['bbox'][1])):
							ff=[pathe,int(v['bbox'][0]),int(v['bbox'][1]),int(v['bbox'][2]),int(v['bbox'][3]),classs]
						if(int(v['bbox'][3])==int(v['bbox'][1])):
							a=True
						if(int(v['bbox'][3])<int(v['bbox'][1])):
							ff=[pathe,int(v['bbox'][0]),int(v['bbox'][3]),int(v['bbox'][2]),int(v['bbox'][1]),classs]
					if(int(v['bbox'][2])==int(v['bbox'][0])):
						a=True
					if(int(v['bbox'][2])<int(v['bbox'][0])):
						if(int(v['bbox'][3])>int(v['bbox'][1])):
							ff=[pathe,int(v['bbox'][2]),int(v['bbox'][1]),int(v['bbox'][0]),int(v['bbox'][3]),classs]
						if(int(v['bbox'][3])==int(v['bbox'][1])):
							a=True
						if(int(v['bbox'][3])<int(v['bbox'][1])):
							ff=[pathe,int(v['bbox'][2]),int(v['bbox'][3]),int(v['bbox'][0]),int(v['bbox'][1]),classs]
					y.append(ff)
	# print(y)



In [0]:
# cleaning the dataset- taking data points which have the images
from os import listdir
from os.path import isfile, join
birdfiles = [f for f in listdir("train/bird") if isfile(join("train/bird", f))]
rabbitfiles = [f for f in listdir("train/rabbit") if isfile(join("train/rabbit", f))]
raccoonfiles = [f for f in listdir("train/raccoon") if isfile(join("train/raccoon", f))]
bobcatfiles = [f for f in listdir("train/bobcat") if isfile(join("train/bobcat", f))]
carfiles = [f for f in listdir("train/car") if isfile(join("train/car", f))]
catfiles = [f for f in listdir("train/cat") if isfile(join("train/cat", f))]
coyotefiles = [f for f in listdir("train/coyote") if isfile(join("train/coyote", f))]
squirrelfiles = [f for f in listdir("train/squirrel") if isfile(join("train/squirrel", f))]

yy=[]
for v in y:
  if(v[5]=="bird"):
    lak=v[0].split('/')
    if(lak[-1] in birdfiles):
      yy.append(v)
  if(v[5]=="rabbit"):
    lak=v[0].split('/')
    if(lak[-1] in rabbitfiles):
      yy.append(v)
  if(v[5]=="raccoon"):
    lak=v[0].split('/')
    if(lak[-1] in raccoonfiles):
      yy.append(v)
  if(v[5]=="bobcat"):
    lak=v[0].split('/')
    if(lak[-1] in bobcatfiles):
      yy.append(v)
  if(v[5]=="car"):
    lak=v[0].split('/')
    if(lak[-1] in carfiles):
      yy.append(v)
  if(v[5]=="cat"):
    lak=v[0].split('/')
    if(lak[-1] in catfiles):
      yy.append(v)
  if(v[5]=="coyote"):
    lak=v[0].split('/')
    if(lak[-1] in coyotefiles):
      yy.append(v)
  if(v[5]=="squirrel"):
    lak=v[0].split('/')
    if(lak[-1] in squirrelfiles):
      yy.append(v)
print(len(y))
print(len(yy))

8798
2105


In [0]:
print(len(birdfiles)+len(squirrelfiles)+len(coyotefiles)+len(catfiles)+len(carfiles)+len(bobcatfiles)+len(raccoonfiles)+len(rabbitfiles))

1969


In [0]:
# creating csv files for training as per the standards set by the precoded model 
filename="annotations.csv"
with open(filename, 'w') as csvfile: 
    csvwriter = csv.writer(csvfile)  
    csvwriter.writerows(yy)

# bird,11
# bobcat,6
# car,33
# cat,16
# raccoon,3
# rabbit,10
# coyote,9
# squirrel,5

rows=[]
a=0
for i in range(len(mainIds)):
  rows.append([mainClasses[i],a])
  a+=1

print(rows)
filename="classes.csv"
with open(filename, 'w') as csvfile: 
    csvwriter = csv.writer(csvfile)  
    csvwriter.writerows(rows)


## splitting train validation set
a0=[]
a1=[]
a2=[]
a3=[]
a4=[]
a5=[]
a6=[]
a7=[]

for i in yy:
  if(i[5]=='car'):
    a2.append(i)
  if(i[5]=='cat'):
    a3.append(i)
  if(i[5]=='bobcat'):
    a1.append(i)
  if(i[5]=='squirrel'):
    a7.append(i)
  if(i[5]=='raccoon'):
    a4.append(i)
  if(i[5]=='rabbit'):
    a5.append(i)
  if(i[5]=='coyote'):
    a6.append(i)
  if(i[5]=='bird'):
    a0.append(i)

yyy_train=[]
yyy_val=[]
yyy_train+=a1[:(80*len(a1)//100)+1]
yyy_val+=a1[(80*len(a1)//100)+1:]
yyy_train+=a2[:(80*len(a2)//100)+1]
yyy_val+=a2[(80*len(a2)//100)+1:]
yyy_train+=a3[:(80*len(a3)//100)+1]
yyy_val+=a3[(80*len(a3)//100)+1:]
yyy_train+=a4[:(80*len(a4)//100)+1]
yyy_val+=a4[(80*len(a4)//100)+1:]
yyy_train+=a5[:(80*len(a5)//100)+1]
yyy_val+=a5[(80*len(a5)//100)+1:]
yyy_train+=a6[:(80*len(a6)//100)+1]
yyy_val+=a6[(80*len(a6)//100)+1:]
yyy_train+=a7[:(80*len(a7)//100)+1]
yyy_val+=a7[(80*len(a7)//100)+1:]
yyy_train+=a0[:(80*len(a0)//100)+1]
yyy_val+=a0[(80*len(a0)//100)+1:]

filename="annotations_train.csv"
with open(filename, 'w') as csvfile: 
    csvwriter = csv.writer(csvfile)  
    csvwriter.writerows(yyy_train)
filename="annotations_val.csv"
with open(filename, 'w') as csvfile: 
    csvwriter = csv.writer(csvfile)  
    csvwriter.writerows(yyy_val)

print(len(yy))
print(len(yyy_train))
print(len(yyy_val))


[['bird', 0], ['bobcat', 1], ['car', 2], ['cat', 3], ['raccoon', 4], ['rabbit', 5], ['coyote', 6], ['squirrel', 7]]
2105
1688
417


In [0]:
retinanet = model.resnet50(num_classes=80, pretrained=True)
retinanet.load_state_dict(torch.load('coco_resnet_50_map_0_335_state_dict.pt'))
# x=retinanet.state_dict().keys()
# print(x)
# print(retinanet)

<All keys matched successfully>

In [0]:
 #modifying last layer of pretrained retinanet and loading weights to retinanet2 
retinanet2 = model.resnet50(num_classes=8, pretrained=True)
feature_size=256
num_anchors=9
num_classes=8
retinanet.classificationModel.output=nn.Conv2d(feature_size, num_anchors * num_classes, kernel_size=3, padding=1)
retinanet.num_classes=8
torch.save(retinanet.state_dict(), 'pretrained_weights_after_modifying_last_layer.pt')
retinanet2.load_state_dict(torch.load('pretrained_weights_after_modifying_last_layer.pt'))
print(retinanet2)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [0]:
# print(retinanet.state_dict()['classificationModel.output.weight'])

In [0]:
# print(retinanet2.state_dict()['classificationModel.output.weight'])

In [0]:
#random  re-assignment of last layer - evaluation
dataset_val = CSVDataset(train_file='annotations_val.csv', class_list='classes.csv',transform=transforms.Compose([Normalizer(), Augmenter(), Resizer()]))
retinanet2 = torch.nn.DataParallel(retinanet2).cuda()
retinanet2.training = False
retinanet2.eval()
retinanet2.module.freeze_bn()
# retinanet2.num_classes=8
use_gpu = True
if use_gpu:
    retinanet2 = retinanet2.cuda()
mAP = csv_eval.evaluate(dataset_val, retinanet2)
print(mAP)

In [0]:
# slicing the weights of last layer of retinanet (pretrained) and loading to retinanet3 and 
retinanet = model.resnet50(num_classes=80, pretrained=True)
retinanet.load_state_dict(torch.load('coco_resnet_50_map_0_335_state_dict.pt'))

pretrained_dict = retinanet.state_dict()
pretrained_dict['classificationModel.output.weight']=retinanet.state_dict()['classificationModel.output.weight'][slice(72)] 
pretrained_dict['classificationModel.output.bias']=retinanet.state_dict()['classificationModel.output.bias'][slice(72)] 


retinanet3 = model.resnet50(num_classes=8, pretrained=True)
retinanet3.load_state_dict(pretrained_dict)
# print(retinanet3.state_dict())



<All keys matched successfully>

In [0]:
# slicing the weights of last layer of retinanet (pretrained) and loading to retinanet3  - evaluation
dataset_val = CSVDataset(train_file='annotations_val.csv', class_list='classes.csv',transform=transforms.Compose([Normalizer(), Augmenter(), Resizer()]))
retinanet3 = torch.nn.DataParallel(retinanet3).cuda()
retinanet3.training = False
retinanet3.eval()
retinanet3.module.freeze_bn()
# retinanet2.num_classes=8
use_gpu = True
if use_gpu:
    retinanet3 = retinanet3.cuda()
mAP = csv_eval.evaluate(dataset_val, retinanet3)
print(mAP)

In [0]:
## training of retinanet_new

dataset_train = CSVDataset(train_file='annotations_train.csv', class_list='classes.csv',transform=transforms.Compose([Normalizer(), Augmenter(), Resizer()]))
dataset_val = CSVDataset(train_file='annotations_val.csv', class_list='classes.csv',transform=transforms.Compose([Normalizer(), Augmenter(), Resizer()]))
sampler = AspectRatioBasedSampler(dataset_train, batch_size=2, drop_last=False)
dataloader_train = DataLoader(dataset_train, num_workers=3, collate_fn=collater, batch_sampler=sampler)
# retinanet - specify model and also change the weights
# retinanet_new = model.resnet50(num_classes=dataset_train.num_classes(), pretrained=True)
#retinanet 2
retinanet_new = model.resnet50(num_classes=8, pretrained=True)
# retinanet_new.load_state_dict(torch.load('pretrained_weights_after_modifying_last_layer.pt'))
retinanet_new=torch.load('retinanet2_9_epochs.pt')
# retinanet_new=retinanet3
# retinanet_new = model.resnet50(num_classes=8, pretrained=True)
# retinanet_new.load_state_dict(pretrained_dict)

#freezing of any layer
#----


use_gpu = True
if use_gpu:
    retinanet_new = retinanet_new.cuda()

retinanet_new = torch.nn.DataParallel(retinanet_new).cuda()
retinanet_new.training = True
optimizer = optim.Adam(retinanet_new.parameters(), lr=(1e-6))
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=3, verbose=True)

loss_hist = collections.deque(maxlen=500)

retinanet_new.train()
retinanet_new.module.freeze_bn()
loss_forplot=[]
for epoch_num in range(5):

    retinanet_new.train()
    retinanet_new.module.freeze_bn()

    epoch_loss = []

    for iter_num, data in enumerate(dataloader_train):
        try:
            optimizer.zero_grad()

            classification_loss, regression_loss = retinanet_new([data['img'].cuda().float(), data['annot']])

            classification_loss = classification_loss.mean()
            regression_loss = regression_loss.mean()

            loss = classification_loss + regression_loss

            if bool(loss == 0):
                continue

            loss.backward()

            torch.nn.utils.clip_grad_norm_(retinanet_new.parameters(), 0.1)

            optimizer.step()

            loss_hist.append(float(loss))

            epoch_loss.append(float(loss))

            print(
                'Epoch: {} | Iteration: {} | Classification loss: {:1.5f} | Regression loss: {:1.5f} | Running loss: {:1.5f}'.format(
                    epoch_num, iter_num, float(classification_loss), float(regression_loss), np.mean(loss_hist)))

            del classification_loss
            del regression_loss
        except Exception as e:
            print(e)
            continue
    print(epoch_num)
    print('Evaluating dataset')
    retinanet_new.eval()
    mAP = csv_eval.evaluate(dataset_val, retinanet_new) 
    scheduler.step(np.mean(epoch_loss))
    loss_forplot.append(np.mean(epoch_loss))
    print(np.mean(epoch_loss))
    print("---------------------------------------")
    torch.save(retinanet_new.module, 'retinanet2_{}_epochs.pt'.format(epoch_num+10))

retinanet_new.eval()
print(loss_forplot)

Epoch: 0 | Iteration: 0 | Classification loss: 0.09807 | Regression loss: 0.09434 | Running loss: 0.19241
Epoch: 0 | Iteration: 1 | Classification loss: 0.08192 | Regression loss: 0.01997 | Running loss: 0.14715
Epoch: 0 | Iteration: 2 | Classification loss: 0.17243 | Regression loss: 0.34511 | Running loss: 0.27061
Epoch: 0 | Iteration: 3 | Classification loss: 0.14315 | Regression loss: 0.28497 | Running loss: 0.30999
Epoch: 0 | Iteration: 4 | Classification loss: 0.01855 | Regression loss: 0.02331 | Running loss: 0.25637
Epoch: 0 | Iteration: 5 | Classification loss: 0.00001 | Regression loss: 0.00057 | Running loss: 0.21374
Epoch: 0 | Iteration: 6 | Classification loss: 0.35384 | Regression loss: 0.20020 | Running loss: 0.26235
Epoch: 0 | Iteration: 7 | Classification loss: 0.23805 | Regression loss: 0.28558 | Running loss: 0.29501
Epoch: 0 | Iteration: 8 | Classification loss: 0.09666 | Regression loss: 0.18526 | Running loss: 0.29356
Epoch: 0 | Iteration: 9 | Classification loss: